In [1]:
import heapq
from collections import deque

def get_successors(state, cap1, cap2):
    x, y = state
    successors = []

    # Fill Jug1
    successors.append(((cap1, y), 'Fill Jug1'))

    # Fill Jug2
    successors.append(((x, cap2), 'Fill Jug2'))

    # Empty Jug1
    successors.append(((0, y), 'Empty Jug1'))

    # Empty Jug2
    successors.append(((x, 0), 'Empty Jug2'))

    # Pour Jug1 -> Jug2
    pour = min(x, cap2 - y)
    successors.append(((x - pour, y + pour), 'Pour Jug1 -> Jug2'))

    # Pour Jug2 -> Jug1
    pour = min(y, cap1 - x)
    successors.append(((x + pour, y - pour), 'Pour Jug2 -> Jug1'))

    return successors

def is_goal(state, goal):
    return goal in state

def bfs(start, cap1, cap2, goal):
    queue = deque([(start, [])])
    visited = set()
    order = []

    while queue:
        (state, path) = queue.popleft()
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for next_state, action in get_successors(state, cap1, cap2):
            if next_state not in visited:
                queue.append((next_state, path + [state]))
    return order, None

def dfs(start, cap1, cap2, goal, visited=None, path=None, order=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []
    if order is None:
        order = []

    order.append(start)
    if is_goal(start, goal):
        return order, path + [start]
    visited.add(start)
    for next_state, _ in get_successors(start, cap1, cap2):
        if next_state not in visited:
            result = dfs(next_state, cap1, cap2, goal, visited, path + [start], order)
            if result[1]:
                return result
    return order, None

def dls(start, cap1, cap2, goal, limit, path=None, order=None):
    if path is None:
        path = []
    if order is None:
        order = []

    order.append(start)
    if is_goal(start, goal):
        return order, path + [start]
    if limit <= 0:
        return order, None

    for next_state, _ in get_successors(start, cap1, cap2):
        if next_state not in path:
            result = dls(next_state, cap1, cap2, goal, limit - 1, path + [start], order)
            if result[1]:
                return result
    return order, None

def ids(start, cap1, cap2, goal, max_depth=20):
    total_order = []
    for depth in range(max_depth):
        order, result = dls(start, cap1, cap2, goal, depth)
        total_order.extend(order)
        if result:
            return total_order, result
    return total_order, None

def ibs(start, cap1, cap2, goal, max_breadth=20):
    visited = set()
    order = []
    queue = deque([(start, [])])

    breadth = 1
    while breadth <= max_breadth:
        next_level = deque()
        level_count = 0
        while queue and level_count < breadth:
            (state, path) = queue.popleft()
            order.append(state)
            if is_goal(state, goal):
                return order, path + [state]
            if state in visited:
                continue
            visited.add(state)
            for next_state, _ in get_successors(state, cap1, cap2):
                if next_state not in visited:
                    next_level.append((next_state, path + [state]))
                    level_count += 1
        queue = next_level
        breadth += 1
    return order, None

def ucs(start, cap1, cap2, goal):
    heap = [(0, start, [])]
    visited = set()
    order = []

    while heap:
        cost, state, path = heapq.heappop(heap)
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for next_state, _ in get_successors(state, cap1, cap2):
            if next_state not in visited:
                heapq.heappush(heap, (cost + 1, next_state, path + [state]))
    return order, None

def heuristic(state, goal):
    return abs(state[0] - goal) + abs(state[1] - goal)

def greedy(start, cap1, cap2, goal):
    heap = [(heuristic(start, goal), start, [])]
    visited = set()
    order = []

    while heap:
        h, state, path = heapq.heappop(heap)
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for next_state, _ in get_successors(state, cap1, cap2):
            if next_state not in visited:
                heapq.heappush(heap, (heuristic(next_state, goal), next_state, path + [state]))
    return order, None

def astar(start, cap1, cap2, goal):
    heap = [(heuristic(start, goal), 0, start, [])]
    visited = set()
    order = []

    while heap:
        f, cost, state, path = heapq.heappop(heap)
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for next_state, _ in get_successors(state, cap1, cap2):
            if next_state not in visited:
                new_cost = cost + 1
                heapq.heappush(heap, (new_cost + heuristic(next_state, goal), new_cost, next_state, path + [state]))
    return order, None

def main():
    cap1 = int(input("Enter capacity of Jug 1: "))
    cap2 = int(input("Enter capacity of Jug 2: "))
    goal = int(input("Enter target amount to measure: "))

    start = (0, 0)

    algorithms = {
        'BFS': bfs,
        'DFS': dfs,
        'DLS (limit=10)': lambda s, a, b, g: dls(s, a, b, g, 10),
        'IDS': ids,
        'IBS': ibs,
        'UCS': ucs,
        'Greedy Best First': greedy,
        'A*': astar
    }

    for name, func in algorithms.items():
        print(f"\n=== {name} ===")
        order, path = func(start, cap1, cap2, goal)
        print("Visited Order:", order)
        print("Solution Path:", path if path else "No solution found")

if __name__ == "__main__":
    main()


Enter capacity of Jug 1:  3
Enter capacity of Jug 2:  4
Enter target amount to measure:  5



=== BFS ===
Visited Order: [(0, 0), (3, 0), (0, 4), (3, 4), (0, 3), (3, 4), (3, 1), (3, 3), (0, 1), (2, 4), (1, 0), (2, 0), (1, 4), (0, 2), (3, 2), (3, 2)]
Solution Path: No solution found

=== DFS ===
Visited Order: [(0, 0), (3, 0), (3, 4), (0, 4), (3, 1), (0, 1), (1, 0), (1, 4), (3, 2), (0, 2), (2, 0), (2, 4), (3, 3), (0, 3)]
Solution Path: No solution found

=== DLS (limit=10) ===
Visited Order: [(0, 0), (3, 0), (3, 0), (3, 4), (3, 4), (0, 4), (0, 4), (3, 1), (3, 1), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 4), (1, 0), (1, 0), (3, 1), (0, 1), (0, 1), (0, 1), (1, 0), (0, 4), (3, 1), (3, 1), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 4), (1, 0), (1, 0), (3, 1), (0, 1), (0, 1), (0, 1), (1, 0), (0, 4), (3, 1), (3, 1), (0, 1), (0, 1), (0, 1), (1, 0), (0, 1), (0, 1), (1, 0), (0, 1), (1, 0), (1, 0), (1, 4), (1, 0), (1, 0), (3, 1), (0, 1), (0, 1), (0, 1), (1, 0), (3, 1), (3, 1), (0, 1), (0, 1), (1, 0), (0, 


KeyboardInterrupt

